In [6]:
import pandas as pd
import re
from datetime import datetime, timedelta

# Функция для преобразования числового значения Excel в формат даты
def excel_to_date(excel_date):
    return datetime(1899, 12, 30) + timedelta(days=excel_date)

# Загрузка данных из таблиц Excel
def load_excel(file_path):
    return pd.read_excel(file_path)

# Исправление ошибок формата даты в колонке 'Дата остаточной стоимости'
def fix_date_format(df):
    if 'Дата остаточной стоимости' in df.columns:
        df['Дата остаточной стоимости'] = df['Дата остаточной стоимости'].apply(lambda x: excel_to_date(x) if isinstance(x, (int, float)) else x)
    return df

# Сравнение двух таблиц и формирование отчета
def compare_tables(old_table, new_table):
    changes = []

    # Поиск новых объектов
    new_objects = set(new_table['РНЗС']) - set(old_table['РНЗС'])
    for obj in new_objects:
        new_obj = new_table[new_table['РНЗС'] == obj].iloc[0]
        address_info = f"{new_obj['Город']}, {new_obj['Страна']}, {new_obj['Адрес']}, {new_obj['Наименование объекта учёта']}, {new_obj['Площадь']}"
        changes.append(f"Добавлен новый объект: {obj}, {address_info}")

    # Поиск выбывших объектов
    removed_objects = set(old_table['РНЗС']) - set(new_table['РНЗС'])
    for obj in removed_objects:
        old_obj = old_table[old_table['РНЗС'] == obj].iloc[0]
        address_info = f"{old_obj['Город']}, {old_obj['Страна']}, {old_obj['Адрес']}, {old_obj['Наименование объекта учёта']}, {old_obj['Площадь']}"
        changes.append(f"Выбыл объект: {obj}, {address_info}")

    # Поиск изменений статуса РНЗС
    for index, row in new_table.iterrows():
        obj_id = row['РНЗС']
        old_row = old_table[old_table['РНЗС'] == obj_id]
        if not old_row.empty:
            if obj_id[0] != old_row.iloc[0]['РНЗС'][0]:
                changes.append(f"Изменение статуса объекта {obj_id}: {old_row.iloc[0]['РНЗС'][0]} -> {obj_id[0]}")

    # Сравнение изменений по существующим объектам
    for index, row in new_table.iterrows():
        obj_id = row['РНЗС']
        old_row = old_table[old_table['РНЗС'] == obj_id]
        if not old_row.empty:
            obj_changes = []
            for column in new_table.columns:
                old_value = old_row.iloc[0][column]
                new_value = row[column]
                if pd.notna(old_value) and pd.notna(new_value) and old_value != new_value:
                    obj_changes.append(f"Колонка '{column}': {old_value} -> {new_value}")
            if obj_changes:
                old_obj = old_row.iloc[0]
                address_info = f"{old_obj['Страна']}, {old_obj['Город']}, {old_obj['Адрес']}, {old_obj['Наименование объекта учёта']}, {old_obj['Площадь']}"
                changes.append(f"Изменение в объекте {obj_id}, ({address_info}): {'; '.join(obj_changes)}")

    return changes

# Генерация отчета на основе изменений
def generate_report(changes):
    if changes:
        report = "Справка по изменениям:\n"
        report += "\n".join(changes)
    else:
        report = "Изменений не обнаружено."
    return report

def generate_changes_table_excel(changes):
    obj_ids = []
    obj_ch=[]
    for change in changes:
        match = re.search(r'([ПВ]\d{11})', change)
        if match:
            obj_id = match.group(1)
            obj_ids.append(obj_id)
    for change in changes:
        match = re.search(r'(Колонка.*)', change) or re.search(r'(Добавлен.*)', change) or re.search(r'(Выбыл.*)', change) or re.search(r'(Изменение.*)', change)
        if match:
            obj_changes = match.group(1)
            obj_ch.append(obj_changes)
    changes_table = pd.DataFrame({'РНЗС':obj_ids,'Изменения':obj_ch})

    # Сохранение таблицы в Excel
    changes_table.to_excel('отчет_изменений.xlsx', index=False)

    return changes_table


# Путь к старой и новой таблице
old_table_path = "/content/Свод (6).xlsx"
new_table_path = "/content/Свод (7).xlsx"

# Загрузка и исправление таблиц
old_table = fix_date_format(load_excel(old_table_path))
new_table = fix_date_format(load_excel(new_table_path))

# Сравнение таблиц и формирование отчета
changes = compare_tables(old_table, new_table)

# Генерация и вывод отчета
report = generate_report(changes)
generate_changes_table_excel(changes)
print(report)

# Сохранение отчета в файл

with open("отчет_по_изменениям.txt", "w") as file:
    file.write(report)


Справка по изменениям:
Изменение в объекте П12890008730, (Австрийская Республика, Вена, 1180, Молнаргассе, 14/Наафгассе, 73, Жилой дом, 867.6): Колонка 'Фактическое использование': Используется правообладателем -> Используется иными лицами и правообладателем; Колонка 'Категория целевого использования': 4 -> 3
Изменение в объекте П11890001909, (Австрийская Республика, Вена, 1180, Нааффгассе, 41, Земельный участок, -): Колонка 'Фактическое использование': Используется правообладателем -> Используется иными лицами и правообладателем
Изменение в объекте П11890001907, (Австрийская Республика, Вена, 1180, Нааффгассе, 41, Земельный участок, -): Колонка 'Фактическое использование': Используется правообладателем -> Используется иными лицами и правообладателем
Изменение в объекте П12890002947, (Австрийская Республика, Вена, 1180, Нааффгассе, 41, Служебно-жилое здание, 216): Колонка 'Фактическое использование': Используется правообладателем -> Используется иными лицами и правообладателем
Изменени